# 사용 패키지

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 데이터 로드

In [2]:
total = pd.read_csv('/content/drive/MyDrive/도시가스/한국가스공사_시간별 공급량_20181231.csv', encoding='cp949')

In [3]:
total.head()

,연월일,시간,구분,공급량
0,2013-01-01,1,A,2497.129
1,2013-01-01,2,A,2363.265
2,2013-01-01,3,A,2258.505
3,2013-01-01,4,A,2243.969
4,2013-01-01,5,A,2344.105


# 전처리

In [4]:
total['구분'].unique()

array(['A', 'B', 'C', 'D', 'E', 'G', 'H'], dtype=object)

In [5]:
d_map = {}
for i, d in enumerate(total['구분'].unique()):
    d_map[d] = i
total['구분'] = total['구분'].map(d_map)

In [6]:
total['연월일'] = pd.to_datetime(total['연월일'])

In [7]:
total['year'] = total['연월일'].dt.year
total['month'] = total['연월일'].dt.month
total['day'] = total['연월일'].dt.day
total['weekday'] = total['연월일'].dt.weekday

In [8]:
total

,연월일,시간,구분,공급량,year,month,day,weekday
0,2013-01-01,1,0,2497.129,2013,1,1,1
1,2013-01-01,2,0,2363.265,2013,1,1,1
2,2013-01-01,3,0,2258.505,2013,1,1,1
3,2013-01-01,4,0,2243.969,2013,1,1,1
4,2013-01-01,5,0,2344.105,2013,1,1,1
...,...,...,...,...,...,...,...,...
368083,2018-12-31,20,6,681.033,2018,12,31,0
368084,2018-12-31,21,6,669.961,2018,12,31,0
368085,2018-12-31,22,6,657.941,2018,12,31,0
368086,2018-12-31,23,6,610.953,2018,12,31,0


In [11]:
outlier_idx = set()
for i in total[total.month == 1].공급량[total[total.month == 1].공급량 >= 10000].index.values:
    outlier_idx.add(i)
for i in total[total.month == 2].공급량[total[total.month == 2].공급량 >= 8000].index.values:
    outlier_idx.add(i)
for i in total[total.month == 4].공급량[total[total.month == 4].공급량 >= 10000].index.values:
    outlier_idx.add(i)
for i in total[total.month == 7].공급량[total[total.month == 7].공급량 >= 3000].index.values:
    outlier_idx.add(i)
for i in total[total.month == 8].공급량[total[total.month == 8].공급량 >= 2500].index.values:
    outlier_idx.add(i)
for i in total[total.month == 11].공급량[total[total.month == 11].공급량 >= 8000].index.values:
    outlier_idx.add(i)

In [12]:
total.drop(outlier_idx, axis=0, inplace=True)

In [13]:
train_years = [2013,2014,2015,2016,2017]
val_years = [2018]

In [14]:
train = total[total['year'].isin(train_years)]
val = total[total['year'].isin(val_years)]

In [15]:
features = ['구분', 'month', 'day', 'weekday', '시간']
train_x = train[features]
train_y = train['공급량']

val_x = val[features]
val_y = val['공급량']

# 학습

In [16]:
d_train = lgb.Dataset(train_x, train_y)
d_val = lgb.Dataset(val_x, val_y)

params = {
    'objective': 'regression',
    'metric':'mae',
    'seed':42
}

model = lgb.train(params, d_train, 1000, d_val, verbose_eval=20, early_stopping_rounds=20)

Training until validation scores don't improve for 10 rounds.
[20]	valid_0's l1: 244.204
[40]	valid_0's l1: 174.889
[60]	valid_0's l1: 158.21
[80]	valid_0's l1: 153.015
[100]	valid_0's l1: 150.791
[120]	valid_0's l1: 149.649
[140]	valid_0's l1: 148.339
[160]	valid_0's l1: 147.782
[180]	valid_0's l1: 147.066
[200]	valid_0's l1: 146.219
Early stopping, best iteration is:
[209]	valid_0's l1: 145.892


# 추론 및 결과 제출

In [9]:
test = pd.read_csv('/content/drive/MyDrive/도시가스/test.csv')
submission = pd.read_csv('/content/drive/MyDrive/도시가스/sample_submission.csv')

In [17]:
test

,일자|시간|구분
0,2019-01-01 01 A
1,2019-01-01 02 A
2,2019-01-01 03 A
3,2019-01-01 04 A
4,2019-01-01 05 A
...,...
15115,2019-03-31 20 H
15116,2019-03-31 21 H
15117,2019-03-31 22 H
15118,2019-03-31 23 H


In [18]:
submission.head()

,일자|시간|구분,공급량
0,2019-01-01 01 A,0
1,2019-01-01 02 A,0
2,2019-01-01 03 A,0
3,2019-01-01 04 A,0
4,2019-01-01 05 A,0


In [19]:
test['일자'] = test['일자|시간|구분'].str.split(' ').str[0]
test['시간'] = test['일자|시간|구분'].str.split(' ').str[1].astype(int)
test['구분'] = test['일자|시간|구분'].str.split(' ').str[2]

In [20]:
test['일자'] = pd.to_datetime(test['일자'])
test['year'] = test['일자'].dt.year
test['month'] = test['일자'].dt.month
test['day'] = test['일자'].dt.day
test['weekday'] = test['일자'].dt.weekday

In [21]:
test['구분'] = test['구분'].map(d_map)

In [22]:
test_x = test[features]

In [23]:
test_x

,구분,month,day,weekday,시간
0,0,1,1,1,1
1,0,1,1,1,2
2,0,1,1,1,3
3,0,1,1,1,4
4,0,1,1,1,5
...,...,...,...,...,...
15115,6,3,31,6,20
15116,6,3,31,6,21
15117,6,3,31,6,22
15118,6,3,31,6,23


In [24]:
preds = model.predict(test_x)

In [25]:
submission['공급량'] = preds

In [26]:
submission.to_csv('/content/drive/MyDrive/도시가스/except_outlier.csv', index=False)